In [1]:
# Import dependencies
import pandas as ps
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras import utils 
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import glob

# Setting random seeds to reduce the amount of randomness in the neural net weights and results
# The results may still not be exactly reproducible
np.random.seed(42)
tf.random.set_seed(42)
import tensorflow as tf
import os
import glob
import numpy as np
from transformers import BertTokenizer, TFBertForMaskedLM
from keras.callbacks import ModelCheckpoint
from transformers import *
import transformers
import keras
import shutil
import time


2023-10-04 15:57:50.700362: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 15:57:50.765727: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-04 15:57:51.035215: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/larkinan/miniconda3/envs/tf/lib/python3.9/site-packages/cv2/../../lib

ninja: no work to do.


In [2]:

class TFRecordReader():
    
    def __init__(self,dataFolder,batchSize):
        self.dataFolder = dataFolder
        self.tfRecords = self.load_dataset(dataFolder,'*.tfrecords')
        self.batch_size = batchSize
        self.AUTOTUNE = tf.data.AUTOTUNE
    
        
    def parse_tfr_element(self,element):
      #use the same structure as above; it's kinda an outline of the structure we now want to create
        data = {
           # 'len': tf.io.FixedLenFeature([], tf.int64),
            'input_ids':tf.io.FixedLenFeature([],tf.string),
           # 'token_type_ids':tf.io.FixedLenFeature([],tf.string),
            'masks':tf.io.FixedLenFeature([],tf.string),
            'labels':tf.io.FixedLenFeature([],tf.string),
        }


        content = tf.io.parse_single_example(element, data)
        print(content.keys())

        #len = content['len']
        input_ids = content['input_ids']
        #token_type_ids = content['token_type_ids']
        masks = content['masks']
        labels = content['labels']

        in_ids = tf.io.parse_tensor(input_ids, out_type=tf.int32)
        #token_ids = tf.io.parse_tensor(token_type_ids, out_type=tf.int64)
        msk = tf.io.parse_tensor(masks, out_type=tf.int32)
        lab = tf.io.parse_tensor(labels, out_type=tf.int64)
        
        return([(in_ids,msk),lab])
    
            

    def load_dataset(self,tfdir,pattern):
        print(tfdir)
        print(len(os.listdir(tfdir)))
        files =  glob.glob(tfdir+pattern,recursive=False)
        a = list(files)
        print(a[0])
        np.random.shuffle(a)
        print("found %i tf records \n\n\n" %(len(a)))
        dataset = tf.data.TFRecordDataset(a)
        ignore_order = tf.data.Options()
        ignore_order.experimental_deterministic = False  # disable order, increase speed

        dataset.with_options(ignore_order)  # uses data as soon as it streams in, rather than in its original order
        print("parsing")

        dataset = dataset.map(self.parse_tfr_element,num_parallel_calls=tf.data.AUTOTUNE)
        print("parsed")
        dataset = dataset.shuffle(4096*10)
        return (dataset)
    
def loadDataset(dataFolder):
    recordReader = TFRecordReader(dataFolder,128) 
    train_data = recordReader.tfRecords.prefetch(buffer_size=AUTOTUNE)
    #val_data = recordReader.tfRecords[1].prefetch(buffer_size=AUTOTUNE)
    train_data = train_data.batch(TRAIN_BATCH_SIZE,deterministic=False)#.cache()#.map(randomFlipMap)
    #val_data = val_data.batch(VAL_BATCH_SIZE)
    print("loaded dataset")
    return(train_data)
    

In [3]:
TRAIN_DATA_FOLDER = "/mnt/h/Aspire/BERT/health/train/"
TEST_DATA_FOLDER = '/mnt/h/Aspire/BERT/health/test/'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))
AUTOTUNE = tf.data.experimental.AUTOTUNE
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 128
#trainData = loadDataset(DATA_FOLDER)
testData = loadDataset(TEST_DATA_FOLDER)

2023-10-04 15:57:53.458859: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/mnt/h/Aspire/BERT/health/test/
1
/mnt/h/Aspire/BERT/health/test/healthDataTest.tfrecords
found 1 tf records 





2023-10-04 15:57:53.461458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-04 15:57:53.461494: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-04 15:57:53.470139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-04 15:57:53.470196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-04 15:57:53.470218: I tensorflow/compile

parsing
dict_keys(['input_ids', 'labels', 'masks'])
parsed
loaded dataset


/home/larkinan/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:1990: UserWarning: The `deterministic` argument has no effect unless the `num_parallel_calls` argument is specified.
  warnings.warn("The `deterministic` argument has no effect unless the "


In [4]:
def getLabels():
    labelSet = []
    for images, labels in testData.take(1000):
        if(len(labelSet)==0):
            labelSet = labels.numpy()
        else:
            labelSet = np.concatenate((labelSet,labels))
    print(labelSet.shape)
    return(labelSet)

In [5]:
def buildClassifictionModel(weightFile,debug=False):
    MAX_SEQ_LEN = 100
    #baseModel = build_base()
    #baseModel.load_weights(weightFile)
    t_i = tf.keras.layers.Input((MAX_SEQ_LEN), dtype=tf.int32, name='input_ids')
    m_i = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype=tf.int32, name='atttention_mask')
    btok = TFBertModel.from_pretrained("/mnt/h/Aspire/BERT/testSaveBERT")#([t_i,m_i])

    # for w in btok.bert.weights:
    #     if(w.name in ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0']):
    #         #print("weights can be trained")
    #         w._trainable = True
    #     else:
    #         #print("weights should be frozen")
    #         w._trainable=False
    # if(debug):
    #     for w in btok.bert.weights:
    #         print("%s is %s" %(w.name,w._trainable))



    a = btok([t_i,m_i])
    a = tf.keras.layers.Dropout(0.25)((a['pooler_output']))
    activation = tf.keras.layers.Dense(132,activation='relu',name='activate1')(a)
    activation = tf.keras.layers.Dropout(0.25)(activation)
    activation = tf.keras.layers.Dense(132,activation='relu',name='activate1')(a)
    activation = tf.keras.layers.Dropout(0.25)(activation)
    #activation = tf.keras.layers.Dense(6,activation='softmax',name='categorical')(a['pooler_output'])
    activation = tf.keras.layers.Dense(5,activation='softmax',name='categorical')(activation)
    #activation = tf.keras.layers.Dense(1,activation='softmax')
    bm2 = tf.keras.Model(inputs = [t_i,m_i],outputs = activation)
    
    return(bm2)

In [6]:
#modelFile = '/mnt/h/Aspire/BERT/child/checkpiont_3layers/age_model_checkpoint.h5'
modelFile = '/mnt/h/Aspire/BERT/health/health_model_checkpoint.h5'
final_BERT = tf.keras.models.load_model(modelFile,custom_objects={"TFBertModel": transformers.TFBertModel})
final_BERT.compile(loss=[tf.keras.losses.BinaryCrossentropy()],optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

Model config BertConfig {
  "_name_or_path": "/mnt/h/Aspire/BERT/testSaveBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30623
}



In [7]:
#BERT_pred = final_BERT.predict(testData)
def getPredictions(dataset,model):
    labelSet,predictions = [],[]
    for text, labels in testData.take(1000):
        pred = model.predict(text)
        if(len(labelSet)==0):
            labelSet = labels.numpy()
            predictions = pred
        else:
            labelSet = np.concatenate((labelSet,labels))
            predictions = np.concatenate((predictions,pred))
    return([labelSet,predictions])
labels,predictions = getPredictions(testData,final_BERT)

4/4 [==============================] - 1s 44ms/step


2023-10-04 15:58:08.640699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 1s 1s/step


In [8]:
print(tf.sigmoid(predictions[8:10]))
predictions2 = tf.round(tf.nn.sigmoid(predictions))
print(predictions2[0:50])
print(predictions2.shape)

tf.Tensor(
[[1.53472371e-04 2.01094002e-02 3.81221715e-03 1.19592564e-03
  3.68692949e-02 3.05096316e-03 9.76612389e-01]
 [6.70108199e-03 1.36743365e-02 2.50349827e-02 1.63421202e-02
  1.97404593e-01 5.92333555e-01 1.31166428e-01]], shape=(2, 7), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0.

In [8]:
def calcTruePositive(labels,predictions):
    predictionsRound = tf.round(tf.nn.sigmoid(predictions))
    correct = np.multiply(labels,predictionsRound)
    nCorrect = np.sum(correct,axis=0)
    percCorrect = nCorrect/np.sum(labels,axis=0)*100
    return(nCorrect,percCorrect)
calcTruePositive(labels,predictions)

(array([ 73., 287., 541., 211., 666.]),
 array([29.79591837, 28.81526104, 65.65533981, 27.7997365 , 61.83844011]))

In [9]:
def calcTrueNegative(labels,predictions):
    predictionsRound = tf.round(tf.nn.sigmoid(predictions))
    labZeros = np.where(labels==0,1,0)
    predZeros = np.where(predictionsRound==0,1,0)
    correct = np.multiply(labZeros,predZeros)
    nCorrect = np.sum(correct,axis=0)
    percCorrect = nCorrect/np.sum(labZeros,axis=0)*100
    return(nCorrect,percCorrect)
calcTrueNegative(labels,predictions)

(array([4726, 3773, 3944, 4090, 3595]),
 array([99.39011567, 94.23076923, 94.44444444, 96.43951898, 91.63905175]))

In [10]:
def calcFalseNegative(labels,predictions):
    predictionsRound = tf.round(tf.nn.sigmoid(predictions))
    predZeros = np.where(predictionsRound==0,1,0)
    correct = np.multiply(labels,predZeros)
    nCorrect = np.sum(correct,axis=0)
    percCorrect = nCorrect/np.sum(labels,axis=0)*100
    return(nCorrect,percCorrect)
calcFalseNegative(labels,predictions)

(array([172, 709, 283, 548, 411]),
 array([70.20408163, 71.18473896, 34.34466019, 72.2002635 , 38.16155989]))

In [11]:
def calcFalsePositive(labels,predictions):
    predictionsRound = tf.round(tf.nn.sigmoid(predictions))
    labZeros = np.where(labels==0,1,0)
    correct = np.multiply(labZeros,predictionsRound)
    nCorrect = np.sum(correct,axis=0)
    print(nCorrect)
    percCorrect = nCorrect/np.sum(labels,axis=0)*100
    return(nCorrect,percCorrect)
calcFalsePositive(labels,predictions)

[ 29. 231. 232. 151. 328.]


(array([ 29., 231., 232., 151., 328.]),
 array([11.83673469, 23.19277108, 28.15533981, 19.89459816, 30.4549675 ]))

In [26]:
labeledData = ps.read_csv("/mnt/h/Aspire/BERT/child/childTweetsStrict.csv",encoding='utf-8',lineterminator='\n')
#labeledData.head()
testData = labeledData[labeledData['test']==1]
trainData = labeledData[labeledData['test']==0]
trainData = trainData.sample(frac=1)
print(trainData.count()[0])
#print(trainData.tail())
trainData = trainData.drop_duplicates()
trainData = trainData.sample(frac=1)
print(trainData.count()[0])
print(sum(trainData['isChild'][-1000:]==1))

82412
82412
277


In [27]:
print(tf.round(tf.nn.sigmoid(predictions[0:10])))

tf.Tensor(
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]], shape=(10, 6), dtype=float32)


In [28]:
def mapNewLineReplace(origLine):
    newLine = origLine.replace("\n",". ")
    return(newLine)

In [29]:
def setupModelInputs(tweetData,debug=False):
    tweetText = list(tweetData['text'])
    tweetText = list(map(mapNewLineReplace,tweetText))
    inputs = TOKEN(tweetText,max_length=100,truncation=True,padding='max_length',return_tensors="tf")
    inp_ids = tf.convert_to_tensor(inputs['input_ids'])
    inputs['input_ids'] = inp_ids
    return(inputs)

In [32]:
TOKEN = BertTokenizer.from_pretrained('expandedTokenBase')
tweetData = testData
modelInputs = setupModelInputs(tweetData)
preds = final_BERT.predict([modelInputs['input_ids'],modelInputs['attention_mask']])
#predictions2 = tf.round(tf.nn.sigmoid(preds))
np.save('/mnt/h/Aspire/BERT/child/test/testPredStrict.npy',tf.nn.sigmoid(preds))
testData.to_csv("/mnt/h/Aspire/BERT/child/test/testDataStrict.csv",encoding='utf-8',index=False)

loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding 😂 to the vocabulary
Adding ❤ to the vocabulary
Adding 😭 to the vocabulary
Adding 😍 to the vocabulary
Adding 🎉 to the vocabulary
Adding 😊 to the vocabulary
Adding 😘 to the vocabulary
Adding 😩 to the vocabulary
Adding 💕 to the vocabulary
Adding 🏽 to the vocabulary
Adding 🏼 to the vocabulary
Adding 🏾 to the vocabulary
Adding 💯 to the vocabulary
Adding 🔥 to the vocabulary
Adding 🏻 to the vocabulary
Adding 🙏 to the vocabulary
Adding 🙌 to the vocabulary
Adding 🤣 to the vocabulary
Adding 😒 to the vocabulary
Adding 💙 to the vocabulary
Adding ♀ to the vocabulary
Adding 🙄 to the vocabulary
Adding 😁 to the vocabulary
Adding 👏 to the vocabulary
Adding ☺ to the vocabulary
Adding 💀 to the vocabulary
Adding 👌 to the vocabulary
Adding 🎈 to the vocabulary
Adding 💜 to the vocabulary
Adding 💗 to the vocabulary
Adding 😅 to the vocabulary
Adding 💖 to the vocabulary
Adding 🤔 t

157/157 [==============================] - 6s 37ms/step


In [33]:
TOKEN = BertTokenizer.from_pretrained('expandedTokenBase')
tweetData = trainData
modelInputs = setupModelInputs(tweetData)
preds = final_BERT.predict([modelInputs['input_ids'],modelInputs['attention_mask']])
predictions2 = tf.round(tf.nn.sigmoid(preds))
np.save('/mnt/h/Aspire/BERT/child/train/trainPredStrict.npy',tf.nn.sigmoid(preds))
trainData.to_csv("/mnt/h/Aspire/BERT/child/train/trainDataStrict.csv",encoding='utf-8',index=False)

loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding 😂 to the vocabulary
Adding ❤ to the vocabulary
Adding 😭 to the vocabulary
Adding 😍 to the vocabulary
Adding 🎉 to the vocabulary
Adding 😊 to the vocabulary
Adding 😘 to the vocabulary
Adding 😩 to the vocabulary
Adding 💕 to the vocabulary
Adding 🏽 to the vocabulary
Adding 🏼 to the vocabulary
Adding 🏾 to the vocabulary
Adding 💯 to the vocabulary
Adding 🔥 to the vocabulary
Adding 🏻 to the vocabulary
Adding 🙏 to the vocabulary
Adding 🙌 to the vocabulary
Adding 🤣 to the vocabulary
Adding 😒 to the vocabulary
Adding 💙 to the vocabulary
Adding ♀ to the vocabulary
Adding 🙄 to the vocabulary
Adding 😁 to the vocabulary
Adding 👏 to the vocabulary
Adding ☺ to the vocabulary
Adding 💀 to the vocabulary
Adding 👌 to the vocabulary
Adding 🎈 to the vocabulary
Adding 💜 to the vocabulary
Adding 💗 to the vocabulary
Adding 😅 to the vocabulary
Adding 💖 to the vocabulary
Adding 🤔 t

2576/2576 [==============================] - 92s 36ms/step


In [ ]:
print(tf.round(predictions2[4990:5000]))

In [ ]:
print(sum(trainData['isChild']))